In [29]:
!pip install -q python-docx faiss-cpu transformers accelerate sentence-transformers

from docx import Document
from google.colab import files
import faiss
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

EMB_MODEL = "silma-ai/silma-embedding-matryoshka-v0.1"
emb_model = SentenceTransformer(EMB_MODEL)

GEN_MODEL = "silma-ai/SILMA-Kashif-2B-Instruct-v1.0"
tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL)
model = AutoModelForCausalLM.from_pretrained(GEN_MODEL, device_map="auto")
rag_pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device_map="auto")

def read_docx(path):
    doc = Document(path)
    return "\n".join(p.text.strip() for p in doc.paragraphs if p.text.strip())

def chunk_text(text, size=800, overlap=200):
    chunks, i, n = [], 0, len(text)
    while i < n:
        j = min(i + size, n)
        chunks.append(text[i:j])
        if j == n: break
        i = max(0, j - overlap)
    return chunks

def build_index(chunks):
    embeds = emb_model.encode(chunks, convert_to_numpy=True, normalize_embeddings=True)
    dim = embeds.shape[1]
    idx = faiss.IndexFlatIP(dim)
    idx.add(embeds)
    return idx

def retrieve(q, idx, chunks, k=5):
    q_emb = emb_model.encode([q], convert_to_numpy=True, normalize_embeddings=True)
    D, I = idx.search(q_emb, k)
    return [chunks[i] for i in I[0] if i < len(chunks)]

def answer_question(question, idx, chunks, top_k=5):
    retrieved = retrieve(question, idx, chunks, top_k)
    context = "\n".join(retrieved)
    prompt = f"السياق:\n{context}\n\nالسؤال: {question}\nالإجابة:"
    out = rag_pipe(prompt, max_new_tokens=150, num_return_sequences=1, do_sample=True)[0]["generated_text"]
    return out.strip()

print("📂 ارفع ملف Word (.docx)")
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

text = read_docx(file_path)
chunks = chunk_text(text, size=1000, overlap=200)
print(f"✅ تم تقسيم النص إلى {len(chunks)} مقطع")

index = build_index(chunks)

questions = [
    "ما هي خطوات التسجيل؟",
    "اشرح متطلبات البرنامج؟"
]

for q in questions:
    print("\n❓ السؤال:", q)
    print("🤖", answer_question(q, index, chunks, top_k=5))



modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

Device set to use cpu


📂 ارفع ملف Word (.docx)


Saving اللائحة الموحدة لبرامج البكالوريوس بنظام الساعات المعتمدة.docx to اللائحة الموحدة لبرامج البكالوريوس بنظام الساعات المعتمدة (6).docx
✅ تم تقسيم النص إلى 61 مقطع

❓ السؤال: ما هي خطوات التسجيل؟
🤖 السياق:
فس شروط التعيين الأول، ويقوم المنسق العام للتحول الرقمي للبرامج بالمهام التالية:
.. مراجعة أعمال التسجيل لجميع البرامج وتدقيقها بعد موافقة المجالس المختصة.
مراجعة أعمال الكنترول واستيفاء مراحل الكنترول النهائية بعد موافقة المجالس المختصة.
الإشراف على متابعة الصحيفة المالية لطلاب البرامج.
مراجعة أعمال الجودة بالبرامج.
مادة [۷]: شروط القيد ومتطلبات الالتحاق
يشترط لقيد الطالب لدرجة البكالوريوس في هذه البرامج بالإضافة إلى الشروط العامة المنصوص عليها في اللائحة التنفيذية
(مادة (٧٥) من قانون تنظيم الجامعات ما يلي:
٤
الباب الأول: القواعد المنظمة اللائحة الموحدة لبرامج البكالوريوس بنظام الساعات المعتمدة
كلية الهندسة - جامعة المنصورة
1. أن يكون الطالب مستوفيا لشروط القبول التي يحددها المجلس الاعلى للجامعات.
. أن يكون الطالب حاصلا على شهادة إتمام الثانوية العامة أو ما يعادلها شعبة رياضيات.